In [17]:
# Extrahiere Tabelle / Philosophennamen aus:
# https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte
#
# Mozilla Firefox shortcuts
# 
# 
# Jupyter Shortcuts: 'H'
#
# https://www.tutorialspoint.com/python_web_scraping/python_modules_for_web_scraping.htm
# https://www.freecodecamp.org/news/how-to-scrape-websites-with-python-2/
# https://oxylabs.io/blog/python-web-scraping
# https://www.freecodecamp.org/news/web-scraping-python-tutorial-how-to-scrape-data-from-a-website/
# https://www.dataquest.io/blog/web-scraping-python-using-beautiful-soup/
#
# https://www.geeksforgeeks.org/parsing-tables-and-xml-with-beautifulsoup/
# https://towardsdatascience.com/a-guide-to-scraping-html-tables-with-pandas-and-beautifulsoup-7fc24c331cf7
#

In [161]:
import sys
import re
import regex
import requests
import csv
from bs4 import BeautifulSoup


In [19]:
web_page_address = "https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte"
print (web_page_address)

https://de.wikipedia.org/wiki/Zeittafel_zur_Philosophiegeschichte


In [20]:
web_page = requests.get(web_page_address)
print ("web_page = \t\t", web_page)
print ("web_page.status_code = \t", web_page.status_code)


web_page = 		 <Response [200]>
web_page.status_code = 	 200


In [21]:
# print ("web_page.content = \t", web_page.content)
# from >timeline.xsd<
event_fieldnames = ("start", "end", "text", "progress", "fuzzy", "locked", "ends_today",
                    "category", "description", "hyperlink", "alert", "icon", "default_color", "milestone")


In [22]:
soup = BeautifulSoup(web_page.content, 'html.parser')
# print(soup.prettify())

In [23]:
bs4_table = soup.find('table', class_='wikitable toptextcells')
# print (type(bs4_table))

In [24]:
def csv_event_file_header_get():
    event_header = ("start", "end", "text", "progress", "fuzzy", "locked", "ends_today", "category", "description", "hyperlink", "alert", "icon", "default_color", "milestone")
    return event_header

In [25]:
def test_write_csv_with_header_file():
    fn = 'csv_tst.csv'
    # with open(sys.argv[1], 'wt') as f:
    with open(fn, 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(csv_event_file_header_get())
        for i in range(3):
            row = (
                i + 1,
                chr(ord('a') + i),
                '08/{:02d}/07'.format(i + 1),
                i,
            )
            writer.writerow(row )

    # print(open(sys.argv[1], 'rt').read())    pass
    print (' -------------------')
    print(open(fn, 'rt').read())

In [26]:
test_write_csv_with_header_file()
def csv_event_make():
    pass

 -------------------
start,end,text,progress,fuzzy,locked,ends_today,category,description,hyperlink,alert,icon,default_color,milestone

1,a,08/01/07,0

2,b,08/02/07,1

3,c,08/03/07,2




In [27]:
def new_d_event_philosopher():
    # return dictionary with keys according to event_fieldnames in >timeline.xsd<
    # all values are preset = ''
    d_event_philosopher = {}
    for cnt, event_fieldname in enumerate (event_fieldnames):
        d_event_philosopher[event_fieldname] = ''
    return d_event_philosopher

In [38]:
def adjust_philosopher_name(td_item):
    philosopher_name = '?'
    if td_item.a:
        philosoph_full_name = td_item.a.string                 # Name
        if ( ' von ' in philosoph_full_name) or  ( ' der ' in philosoph_full_name):
            philosopher_name = philosoph_full_name
        else:
            philosopher_name = philosoph_full_name.split(' ')[-1]
            if philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ', '
            for ele in philosoph_full_name.split(' ')[:-1]:
                philosopher_name = philosopher_name + ele + ' '
    return philosopher_name

In [77]:
rgx_DC = re.compile('\d{0,4}-\d{0,4}')

In [164]:
def adjust_philosoph_lifespan (philosoph_lifespan, d_event_philosopher):
    philosoph_lifespan = philosoph_lifespan.strip()
    # rgx_DC = re.compile('\d{0,4}-\d{0,4}')
    # rgx_DC = re.compile('-\d{4}', flags=re.LOCALE)
    # rgx_DC = re.compile(r"\d{1,4}-\d{1,4}")
    # rgx_DC = re.compile("\d{1,4}-")
    # if rgx_DC.search(philosoph_lifespan):
    if regex.search("\d{1,4}", philosoph_lifespan):
        print (philosoph_lifespan, '  =>  ', philosoph_lifespan)
    return philosoph_lifespan, d_event_philosopher['fuzzy']

In [165]:
row_cnt = 0

act_h2  = ''   # Epoche
act_h3  = ''   # Unterepoche
act_h4  = ''   # Unter-Unterepoche

old_h2  = ''
old_h3  = ''
old_h4  = ''


# lo == list of : event == philosopher
lo_d_event_philosopher  = []
d_event_philosopher     = new_d_event_philosopher()
d_event_philosoph_cnt   = 0

# cycle rows in table:
for bs4_row in bs4_table.find_all('tr'):
    # print (type(bs4_row))   # type == <class 'bs4.element.Tag'>
    bs4_row_text = BeautifulSoup(bs4_row.get_text())  # get_text() == 'converts' type to string, sort of
    row_cnt += 1
    # https://stackoverflow.com/questions/13853025/is-there-a-way-in-beautiful-soup-to-count-the-number-of-tags-in-a-html-page
    td_columns = len (bs4_row.find_all('td'))

    # 1 column in row == Epoche / Unterepoche / Unter-Unterepoche == Headline of span
    if (td_columns == 1):
        if bs4_row.find_all('h2'):
            try:
                act_h2 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2
            except:  pass
        elif bs4_row.find_all('h3'):
            try:
                act_h3 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2 + ' - ' + act_h3
            except: pass
        elif bs4_row.find_all('h4'):
            try:
                act_h4 = bs4_row.find_all('span', class_='mw-headline')[0].string
                category = act_h2  + ' - ' + act_h3  + ' - ' + act_h4
            except: pass
        # print ('category =  ', category)

    elif (td_columns == 4) :
        philosoph_lifespan      = None

        # cycle 4 columns in row:  'Periode' / 'Philosoph' (Name) / 'Philosophie' (Thesen) / 'Allgemeine Geschichte'
        for td_cnt, td_item in enumerate (bs4_row.find_all('td')):
            if   (td_cnt == 0) and (td_item.string):                               #  'Periode'
                philosoph_lifespan = adjust_philosoph_lifespan (td_item.string, d_event_philosopher)
            elif (td_cnt == 1) and (td_item.a.string):                             #  'Philosoph' (Name)
                philosopher.name = adjust_philosopher_name(td_item)
                d_event_philosoph_cnt += 1
                try:
                    philosoph_lifespan = philosoph_lifespan.string.replace("\n", " ")
                except:
                    philosoph_lifespan = '???? '
            elif (td_cnt == 2) and (td_item):                                      #  'Philosophie' (Thesen)
                philosopher_lo_theories = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_theories.append(li_item.text)
            elif (td_cnt == 3) and (td_item):                                      #  'Allgemeine Geschichte'
                # philosopher_lo_history = td_item.text
                philosopher_lo_history = []
                for li_cnt, li_item in enumerate (td_item.find_all('li')):
                    philosopher_lo_history.append(li_item.text)

                line_new = '{:>35}      {}    {}  '.format(philosoph_lifespan, philosopher.name, philosopher_lo_theories)
                # print (d_event_philosoph_cnt, ': ', line_new)

                philosopher.lifespan = philosoph_lifespan


    lo_d_event_philosopher.append(philosopher)



um 624–546   =>   um 624–546
um 610–547   =>   um 610–547
um 585–525   =>   um 585–525
um 499–428   =>   um 499–428
um 570–470   =>   um 570–470
um 515–445   =>   um 515–445
um 490–430   =>   um 490–430
um 490–430   =>   um 490–430
um 540–480   =>   um 540–480
um 499–428   =>   um 499–428
um 494–434   =>   um 494–434
um 580–500   =>   um 580–500
um 500   =>   um 500
um 500   =>   um 500
um 470–399   =>   um 470–399
um 428–347   =>   um 428–347
um 400–335   =>   um 400–335
5. Jh.   =>   5. Jh.
460–371   =>   460–371
5.–4. Jh.   =>   5.–4. Jh.
360–320   =>   360–320
um 490–411   =>   um 490–411
480–411   =>   480–411
um 480–380   =>   um 480–380
um 480–380   =>   um 480–380
um 465–399   =>   um 465–399
5. Jh.   =>   5. Jh.
unsicher (5. Jh.)   =>   unsicher (5. Jh.)
um 460–403   =>   um 460–403
um 450   =>   um 450
um 436–338   =>   um 436–338
† um 375   =>   † um 375
um 400 bis um 350   =>   um 400 bis um 350
469–399   =>   469–399
427–347   =>   427–347
um 384–322   =>   um 384–322
um 4